# AutoGluon Text: Multilingual Text Classification

## Objective
This notebook demonstrates **multilingual text classification** using AutoGluon. The model can handle text in multiple languages without requiring language-specific preprocessing.

## Use Case
Multilingual text classification is useful for:
- Global customer support (multi-language ticket classification)
- International social media sentiment analysis
- Cross-language content moderation
- Multi-market product review analysis
- Language-agnostic document categorization

## Key Features
- Uses multilingual transformer models (e.g., mBERT, XLM-RoBERTa)
- No need for translation or language detection
- Handles code-switching and mixed-language text

In [ ]:
!pip install -q torch torchvision torchaudio
!pip install -q autogluon

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
import os
import shutil

In [ ]:
# Load dataset
# TODO: Upload your multilingual text dataset or use URL
# Your dataset should contain text in multiple languages
# Example columns: 'text' (in various languages), 'label' (category)

# Example: train_data = TabularDataset('https://your-multilingual-dataset-url.csv')

# Example placeholder - replace with your actual data
# train_data = TabularDataset('path/to/your/multilingual_data.csv')
# test_data = TabularDataset('path/to/your/test_data.csv')

train_data = None  # Replace with your data
test_data = None   # Replace with your data

print("Dataset loaded successfully!")
if train_data is not None:
    print(f"Training data shape: {train_data.shape}")
    print("\nSample data (showing different languages):")
    print(train_data.head())

In [ ]:
# Set label column
LABEL = 'label'  # TODO: Replace with your label column name

In [ ]:
# Auto-detect problem type based on label
if train_data is not None and LABEL in train_data.columns:
    # Check if the label is numeric (regression) or categorical (classification)
    if pd.api.types.is_numeric_dtype(train_data[LABEL]):
        # Check if it's continuous or discrete
        unique_ratio = train_data[LABEL].nunique() / len(train_data)
        if unique_ratio > 0.05:  # More than 5% unique values suggests regression
            problem_type = 'regression'
            eval_metric = 'rmse'
        else:
            problem_type = 'classification'
            eval_metric = 'roc_auc'
    else:
        problem_type = 'classification'
        eval_metric = 'roc_auc'
else:
    # Default to classification for multilingual text tasks
    problem_type = 'classification'
    eval_metric = 'roc_auc'

print(f"Problem Type: {problem_type}")
print(f"Evaluation Metric: {eval_metric}")

In [ ]:
# Train the model with multilingual support
# AutoGluon will automatically use multilingual models when text is detected
predictor = TabularPredictor(
    label=LABEL,
    problem_type=problem_type,
    eval_metric=eval_metric,
    path='./autogluon-multilingual-model'
).fit(
    train_data=train_data,
    presets='medium_quality',
    time_limit=900,
    # AutoGluon will automatically select multilingual models
    # like xlm-roberta-base for multilingual text
)

print("Model training completed!")
print("The model can now handle text in multiple languages.")

In [ ]:
# Display and save leaderboard
leaderboard = predictor.leaderboard(test_data, silent=True)
print("\nModel Leaderboard:")
print(leaderboard)

# Save leaderboard to CSV
leaderboard.to_csv('leaderboard.csv', index=False)
print("\nLeaderboard saved to leaderboard.csv")

In [ ]:
# Display and save feature importance
try:
    feature_importance = predictor.feature_importance(test_data)
    print("\nFeature Importance:")
    print(feature_importance)
    
    # Save feature importance to CSV
    feature_importance.to_csv('feature_importance.csv')
    print("\nFeature importance saved to feature_importance.csv")
except Exception as e:
    print(f"Could not compute feature importance: {e}")

In [ ]:
# Make predictions
if test_data is not None:
    predictions = predictor.predict(test_data)
    print("\nPredictions:")
    print(predictions.head())
    
    # For classification, also show prediction probabilities
    if problem_type == 'classification':
        pred_probs = predictor.predict_proba(test_data)
        print("\nPrediction Probabilities:")
        print(pred_probs.head())
        
    # Example: Predict on multilingual text
    print("\nExample predictions on different languages:")
    print("(Replace with your own multilingual text for testing)")
    # multilingual_examples = pd.DataFrame({
    #     'text': [
    #         'This product is amazing!',  # English
    #         'Ce produit est incroyable!',  # French
    #         'この製品は素晴らしい！',  # Japanese
    #         '¡Este producto es increíble!'  # Spanish
    #     ]
    # })
    # print(predictor.predict(multilingual_examples))

In [ ]:
# Save model artifacts as zip file
model_path = './autogluon-multilingual-model'
zip_filename = 'autogluon_multilingual_model'

if os.path.exists(model_path):
    shutil.make_archive(zip_filename, 'zip', model_path)
    print(f"\nModel artifacts saved to {zip_filename}.zip")
else:
    print("Model path not found. Train the model first.")